## Setting up the notebook

In [ ]:
# installing packages
!pip install pip==24.0
!pip install numpy==1.23.5
!pip install tensorboardX
!pip install subword-nmt
!pip install sentencepiece
!pip install wandb

  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl.metadata (5.8 kB)
Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl (101 kB)


In [ ]:
# importing packages
import numpy
import os
import tensorboardX
import sentencepiece as spm
import wandb

In [ ]:
# setting up wandb for logging model performance during training
# api key = b5b05b603ec81167154bcdaec184b83d9e96049b
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
source_code = 'eng'
target_code = 'nde'

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}')

# installing fairseq
#!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
!pip install --editable ./

/content/drive/MyDrive/Research/eng-to-nde/fairseq
Obtaining file:///content/drive/MyDrive/Research/eng-to-nde/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 6.6 MB/s eta 0:00:00
  Building editable for fairseq (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-0.editable-cp310-cp310-linux_x86_64.whl size=9580 sha256=7e825927c0e4c57e2dfcfaf3c85965d35c5c809908c6a7033ea5cff30cd9e6

## Training NMT model with Byte-Pair Encoding

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe')

In [ ]:
!fairseq-train data-bin \
--arch transformer \
--activation-fn relu \
--share-decoder-input-output-embed \
--share-all-embeddings \
--encoder-layers 3 \
--encoder-attention-heads 4 \
--encoder-embed-dim 256 \
--encoder-ffn-embed-dim 1024 \
--decoder-layers 3 \
--decoder-attention-heads 4 \
--decoder-embed-dim 256 \
--decoder-ffn-embed-dim 1024 \
--dropout 0.25 \
--seed 2024 \
--optimizer 'adam' \
--adam-betas '(0.9, 0.999)' \
--lr-scheduler 'inverse_sqrt' \
--patience 5 \
--warmup-updates 1000 \
--criterion 'label_smoothed_cross_entropy' \
--label-smoothing 0.1 \
--lr 0.0003 \
--weight-decay 0.0 \
--max-tokens 4096 \
--max-tokens-valid 3600 \
--required-batch-size-multiple 1 \
--best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
--max-epoch 100 \
--validate-interval 25 \
--save-interval 25 \
--validate-interval-updates 7000 \
--save-interval-updates 7000 \
--log-interval 100 \
--curriculum 0 \
--no-epoch-checkpoints \
--eval-bleu \
--eval-bleu-args '{"beam": 5, "max_len_a": 0, "max_len_b": 100, "len_pen": 1}' \
--eval-bleu-detok space \
--eval-bleu-remove-bpe \
--save-dir checkpoints-bpe \
--ddp-backend=no_c10d \
--wandb-project 'fairseq-standard-subword-tok-eng-to-nde'

2024-11-06 15:24:41.542920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 15:24:41.563214: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 15:24:41.569578: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 15:24:41.585904: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 15:24:42.746124: W tensorflow/comp

## Training NMT with Unigram Language Model

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm')

In [ ]:
!fairseq-train data-bin \
--arch transformer \
--activation-fn relu \
--share-decoder-input-output-embed \
--share-all-embeddings \
--encoder-layers 3 \
--encoder-attention-heads 4 \
--encoder-embed-dim 256 \
--encoder-ffn-embed-dim 1024 \
--decoder-layers 3 \
--decoder-attention-heads 4 \
--decoder-embed-dim 256 \
--decoder-ffn-embed-dim 1024 \
--dropout 0.25 \
--seed 2024 \
--optimizer 'adam' \
--adam-betas '(0.9, 0.999)' \
--lr-scheduler 'inverse_sqrt' \
--patience 5 \
--warmup-updates 1000 \
--criterion 'label_smoothed_cross_entropy' \
--label-smoothing 0.1 \
--lr 0.0003 \
--weight-decay 0.0 \
--max-tokens 4096 \
--max-tokens-valid 3600 \
--required-batch-size-multiple 1 \
--best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
--max-epoch 100 \
--validate-interval 25 \
--save-interval 25 \
--validate-interval-updates 7000 \
--save-interval-updates 7000 \
--log-interval 100 \
--curriculum 0 \
--no-epoch-checkpoints \
--eval-bleu \
--eval-bleu-args '{"beam": 5, "max_len_a": 0, "max_len_b": 100, "len_pen": 1}' \
--eval-bleu-detok space \
--eval-bleu-remove-bpe sentencepiece \
--save-dir checkpoints-ulm \
--ddp-backend=no_c10d \
--wandb-project 'fairseq-standard-subword-tok-eng-to-nde'

Streaming output truncated to the last 5000 lines.
2024-11-06 13:39:25 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.

epoch 037 | valid on 'valid' subset:  22% 57/262 [00:31<02:20,  1.46it/s]
epoch 037 | valid on 'valid' subset:  22% 58/262 [00:32<02:11,  1.55it/s]2024-11-06 13:39:26 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-11-06 13:39:26 | WARNING | sacrebleu | It looks like you forgot to detokenize your test data, which may hurt your score.
2024-11-06 13:39:26 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.

epoch 037 | valid on 'valid' subset:  23% 59/262 [00:33<02:19,  1.45it/s]2024-11-06 13:39:27 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-11-06 13:39:27 | WARNING | sacrebleu | It looks like you forgot to detokenize your tes

## Training NMT with BPE-Dropout

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpeDROP')

In [ ]:
!fairseq-train data-bin-25 \
--arch transformer \
--activation-fn relu \
--share-decoder-input-output-embed \
--share-all-embeddings \
--encoder-layers 3 \
--encoder-attention-heads 4 \
--encoder-embed-dim 256 \
--encoder-ffn-embed-dim 1024 \
--decoder-layers 3 \
--decoder-attention-heads 4 \
--decoder-embed-dim 256 \
--decoder-ffn-embed-dim 1024 \
--dropout 0.25 \
--seed 2024 \
--optimizer 'adam' \
--adam-betas '(0.9, 0.999)' \
--lr-scheduler 'inverse_sqrt' \
--patience 5 \
--warmup-updates 1000 \
--criterion 'label_smoothed_cross_entropy' \
--label-smoothing 0.1 \
--lr 0.0003 \
--weight-decay 0.0 \
--max-tokens 4096 \
--max-tokens-valid 3600 \
--required-batch-size-multiple 1 \
--best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
--max-epoch 4 \
--validate-interval 1 \
--save-interval 1 \
--validate-interval-updates 7000 \
--save-interval-updates 7000 \
--log-interval 100 \
--curriculum 0 \
--no-epoch-checkpoints \
--eval-bleu \
--eval-bleu-args '{"beam": 5, "max_len_a": 0, "max_len_b": 100, "len_pen": 1}' \
--eval-bleu-detok space \
--eval-bleu-remove-bpe \
--save-dir checkpoints-bpeDROP \
--ddp-backend=no_c10d \
--wandb-project 'fairseq-standard-subword-tok-eng-to-nde'

Streaming output truncated to the last 5000 lines.
2024-11-05 12:27:47 | WARNING | sacrebleu | It looks like you forgot to detokenize your test data, which may hurt your score.
2024-11-05 12:27:47 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.

epoch 001 | valid on 'valid' subset:  18% 45/257 [00:26<02:33,  1.38it/s]2024-11-05 12:27:48 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-11-05 12:27:48 | WARNING | sacrebleu | It looks like you forgot to detokenize your test data, which may hurt your score.
2024-11-05 12:27:48 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.

epoch 001 | valid on 'valid' subset:  18% 46/257 [00:27<02:38,  1.33it/s]2024-11-05 12:27:49 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-11-05 12:27:49 | WARNING | sacrebl

## Training NMT with ULM with Subword Regularization

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulmSR')

In [ ]:
!fairseq-train data-bin-25 \
--arch transformer \
--activation-fn relu \
--share-decoder-input-output-embed \
--share-all-embeddings \
--encoder-layers 3 \
--encoder-attention-heads 4 \
--encoder-embed-dim 256 \
--encoder-ffn-embed-dim 1024 \
--decoder-layers 3 \
--decoder-attention-heads 4 \
--decoder-embed-dim 256 \
--decoder-ffn-embed-dim 1024 \
--dropout 0.25 \
--seed 2024 \
--optimizer 'adam' \
--adam-betas '(0.9, 0.999)' \
--lr-scheduler 'inverse_sqrt' \
--patience 5 \
--warmup-updates 1000 \
--criterion 'label_smoothed_cross_entropy' \
--label-smoothing 0.1 \
--lr 0.0003 \
--weight-decay 0.0 \
--max-tokens 4096 \
--max-tokens-valid 3600 \
--required-batch-size-multiple 1 \
--best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
--max-epoch 4 \
--validate-interval 1 \
--save-interval 1 \
--validate-interval-updates 7000 \
--save-interval-updates 7000 \
--log-interval 100 \
--curriculum 0 \
--no-epoch-checkpoints \
--eval-bleu \
--eval-bleu-args '{"beam": 5, "max_len_a": 0, "max_len_b": 100, "len_pen": 1}' \
--eval-bleu-detok space \
--eval-bleu-remove-bpe sentencepiece \
--save-dir checkpoints-ulmSR \
--ddp-backend=no_c10d \
--wandb-project 'fairseq-standard-subword-tok-eng-to-nde'

Streaming output truncated to the last 5000 lines.

epoch 002 | valid on 'valid' subset:  31% 81/262 [01:02<02:17,  1.32it/s]2024-11-06 17:06:40 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-11-06 17:06:40 | WARNING | sacrebleu | It looks like you forgot to detokenize your test data, which may hurt your score.
2024-11-06 17:06:40 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.

epoch 002 | valid on 'valid' subset:  31% 82/262 [01:03<02:19,  1.29it/s]2024-11-06 17:06:41 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-11-06 17:06:41 | WARNING | sacrebleu | It looks like you forgot to detokenize your test data, which may hurt your score.
2024-11-06 17:06:41 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.

epoch 002 | valid on 'valid' subset:  

## Training NMT with Task-Specific Tokenizer

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok')

In [ ]:
!fairseq-train data-bin-nmt \
--arch transformer \
--activation-fn relu \
--share-decoder-input-output-embed \
--share-all-embeddings \
--encoder-layers 3 \
--encoder-attention-heads 4 \
--encoder-embed-dim 256 \
--encoder-ffn-embed-dim 1024 \
--decoder-layers 3 \
--decoder-attention-heads 4 \
--decoder-embed-dim 256 \
--decoder-ffn-embed-dim 1024 \
--dropout 0.25 \
--seed 2024 \
--optimizer 'adam' \
--adam-betas '(0.9, 0.999)' \
--lr-scheduler 'inverse_sqrt' \
--patience 5 \
--warmup-updates 1000 \
--criterion 'label_smoothed_cross_entropy' \
--label-smoothing 0.1 \
--lr 0.0003 \
--weight-decay 0.0 \
--max-tokens 4096 \
--max-tokens-valid 3600 \
--required-batch-size-multiple 1 \
--best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
--max-epoch 100 \
--validate-interval 25 \
--save-interval 25 \
--validate-interval-updates 7000 \
--save-interval-updates 7000 \
--log-interval 100 \
--curriculum 0 \
--no-epoch-checkpoints \
--eval-bleu \
--eval-bleu-args '{"beam": 5, "max_len_a": 0, "max_len_b": 100, "len_pen": 1}' \
--eval-bleu-detok space \
--eval-bleu-remove-bpe sentencepiece \
--save-dir checkpoints-task-nmt \
--ddp-backend=no_c10d \
--wandb-project 'fairseq-standard-subword-tok-eng-to-nde'

Streaming output truncated to the last 5000 lines.
2024-11-05 10:02:28 | WARNING | sacrebleu | It looks like you forgot to detokenize your test data, which may hurt your score.
2024-11-05 10:02:28 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.

epoch 037 | valid on 'valid' subset:  21% 55/262 [00:29<02:26,  1.42it/s]2024-11-05 10:02:29 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-11-05 10:02:29 | WARNING | sacrebleu | It looks like you forgot to detokenize your test data, which may hurt your score.
2024-11-05 10:02:29 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.

epoch 037 | valid on 'valid' subset:  21% 56/262 [00:30<02:33,  1.35it/s]2024-11-05 10:02:30 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-11-05 10:02:30 | WARNING | sacrebl